<a href="https://colab.research.google.com/github/Naman-27072004/RockShield---AI-Copyright-Code-/blob/main/RockShield_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is a complete ML workflow that trains a Random Forest classifier to predict landslide risk, saves the model, and demonstrates how to make predictions.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
from google.colab import files  # only if using Colab

# --- Step 1: Upload your CSV if not already on server ---
uploaded = files.upload()  # This will prompt you to upload files

# Assuming you uploaded "landslide_merged_demo.csv"
csv_filename = "landslide_merged_demo.csv"
df = pd.read_csv(csv_filename)

TARGET = "label"
X = df.drop(columns=["timestamp", TARGET])
y = df[TARGET]

# --- Step 2: Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Step 3: Pipeline setup ---
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=12,
        random_state=42,
        class_weight="balanced"
    ))
])

# --- Step 4: Train model ---
pipeline.fit(X_train, y_train)

print("Train Accuracy:", pipeline.score(X_train, y_train))
print("Test Accuracy :", pipeline.score(X_test, y_test))

# --- Step 5: Save model ---
joblib.dump(pipeline, "rf_landslide_classifier.pkl")
print("✅ Model saved as rf_landslide_classifier.pkl")

# --- Step 6: Prediction helper ---
class_labels = {
    0: "Stable",
    1: "High Risk",
    2: "Landslide Event"
}

def predict_risk(input_dict):
    df_input = pd.DataFrame([input_dict])

    # Ensure all features are present
    for col in X.columns:
        if col not in df_input:
            df_input[col] = np.nan

    df_input = df_input[X.columns]  # reorder columns

    pred = pipeline.predict(df_input)[0]
    proba = pipeline.predict_proba(df_input)[0]

    pred_label = class_labels[pred]
    proba_dict = {class_labels[c]: p for c, p in zip(pipeline.classes_, proba)}

    return pred_label, proba_dict

# --- Step 7: Example prediction ---
example_input = {
    "temperature_C": 14.868,
    "rainfall_mm": 6.946,
    "pitch_deg": 2.4122,
    "soil_moisture": 0.22,
    "roll_deg": 0.9999,
    "crack_mm": 0.49916,
    "crack_growth_mm_per_hr": 0.00017,
    "accel_x_g": -0.03961,
    "accel_y_g": 0.01297,
    "accel_z_g": 0.99266,
    "vibration_rms_g": 0.04279,
    "vibration_peak_g": 0.02472
}

prediction, probabilities = predict_risk(example_input)
print("\nExample Prediction:")
print("Predicted Class:", prediction)
print("Class Probabilities:", probabilities)


Saving landslide_merged_demo.csv to landslide_merged_demo.csv
Train Accuracy: 0.91875
Test Accuracy : 0.87525
✅ Model saved as rf_landslide_classifier.pkl

Example Prediction:
Predicted Class: High Risk
Class Probabilities: {'Stable': np.float64(0.1874943912054663), 'High Risk': np.float64(0.6624961462947841), 'Landslide Event': np.float64(0.15000946249974956)}


This script is a complete ML workflow that trains a Random Forest classifier to predict landslide risk, saves the model, and demonstrates how to make predictions.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# --- Step 1: Load CSV ---
csv_filename = "landslide_merged_demo.csv"   # make sure this file is in your project root
df = pd.read_csv(csv_filename)

TARGET = "label"
X = df.drop(columns=["timestamp", TARGET])
y = df[TARGET]

# --- Step 2: Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Step 3: Pipeline setup ---
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=12,
        random_state=42,
        class_weight="balanced"
    ))
])

# --- Step 4: Train model ---
pipeline.fit(X_train, y_train)

print("Train Accuracy:", pipeline.score(X_train, y_train))
print("Test Accuracy :", pipeline.score(X_test, y_test))

# --- Step 5: Class labels ---
class_labels = {
    0: "Stable",
    1: "High Risk",
    2: "Landslide Event"
}

# --- Step 6: Prediction helper ---
def predict_risk(input_dict):
    df_input = pd.DataFrame([input_dict])

    # Ensure all features are present
    for col in X.columns:
        if col not in df_input:
            df_input[col] = np.nan

    df_input = df_input[X.columns]  # reorder columns

    pred = pipeline.predict(df_input)[0]
    proba = pipeline.predict_proba(df_input)[0]

    pred_label = class_labels[pred]
    proba_dict = {class_labels[c]: float(p) for c, p in zip(pipeline.classes_, proba)}

    return pred_label, proba_dict

# --- Step 7: Example prediction ---
example_input = {
    "temperature_C": 14.868,
    "rainfall_mm": 6.946,
    "pitch_deg": 2.4122,
    "soil_moisture": 0.22,
    "roll_deg": 0.9999,
    "crack_mm": 0.49916,
    "crack_growth_mm_per_hr": 0.00017,
    "accel_x_g": -0.03961,
    "accel_y_g": 0.01297,
    "accel_z_g": 0.99266,
    "vibration_rms_g": 0.04279,
    "vibration_peak_g": 0.02472
}

prediction, probabilities = predict_risk(example_input)
print("\nExample Prediction:")
print("Predicted Class:", prediction)
print("Class Probabilities:", probabilities)


Train Accuracy: 0.91875
Test Accuracy : 0.87525

Example Prediction:
Predicted Class: High Risk
Class Probabilities: {'Stable': 0.1874943912054663, 'High Risk': 0.6624961462947841, 'Landslide Event': 0.15000946249974956}


In [3]:
!pip install djangorestframework


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 106.1 MB/s eta 0:00:00


This code defines a REST API endpoint (/predict) for predicting landslide risk levels based on environmental sensor readings — such as temperature, rainfall, and soil moisture.
It trains a Random Forest machine learning model automatically when the server starts, then uses it to make predictions for incoming POST requests.(THIS CODE IS NOT FOR COLAB)

In [4]:
# # views.py

# from rest_framework.decorators import api_view
# from rest_framework.response import Response
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# import os

# # --- Load CSV and train model at startup ---
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# CSV_PATH = os.path.join(BASE_DIR, "landslide_merged_demo.csv")

# # Load dataset
# df = pd.read_csv(CSV_PATH)

# # Define target and features
# TARGET = "label"
# X = df.drop(columns=["timestamp", TARGET])
# y = df[TARGET]

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # ML Pipeline: Imputation → Scaling → RandomForest
# pipeline = Pipeline([
#     ("imputer", SimpleImputer(strategy="mean")),
#     ("scaler", StandardScaler()),
#     ("model", RandomForestClassifier(
#         n_estimators=200,
#         max_depth=12,
#         random_state=42,
#         class_weight="balanced"
#     ))
# ])

# # Train model once at server startup
# pipeline.fit(X_train, y_train)

# print("✅ Model trained at startup")
# print("Train Accuracy:", pipeline.score(X_train, y_train))
# print("Test Accuracy :", pipeline.score(X_test, y_test))

# # Label mapping
# class_labels = {
#     0: "Stable",
#     1: "High Risk",
#     2: "Landslide Event"
# }

# # --- Helper: Predict risk from input JSON ---
# def predict_risk(input_dict):
#     df_input = pd.DataFrame([input_dict])

#     # Ensure all expected features exist
#     for col in X.columns:
#         if col not in df_input:
#             df_input[col] = np.nan

#     # Keep same feature order
#     df_input = df_input[X.columns]

#     # Run model
#     pred = pipeline.predict(df_input)[0]
#     proba = pipeline.predict_proba(df_input)[0]

#     # Convert to readable output
#     pred_label = class_labels[pred]
#     proba_dict = {class_labels[c]: float(p) for c, p in zip(pipeline.classes_, proba)}

#     return pred_label, proba_dict

# # --- REST API Endpoint ---
# @api_view(['POST'])
# def predict(request):
#     """
#     POST /predict
#     Input: JSON with feature values
#     Output: Predicted risk + probabilities
#     """
#     input_data = request.data  # Expect JSON payload
#     prediction, probabilities = predict_risk(input_data)

#     return Response({
#         "prediction": prediction,
#         "probabilities": probabilities
#     })


In [5]:
# --- ML Landslide Risk Prediction (Colab Version) ---

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os

# Path to CSV (upload it or mount drive)
CSV_PATH = "/content/landslide_merged_demo.csv"

# --- Load and prepare data ---
df = pd.read_csv(CSV_PATH)

TARGET = "label"
X = df.drop(columns=["timestamp", TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- ML pipeline ---
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("model", RandomForestClassifier(
        n_estimators=200,
        max_depth=12,
        random_state=42,
        class_weight="balanced"
    ))
])

# Train
pipeline.fit(X_train, y_train)

print("✅ Model trained")
print("Train Accuracy:", pipeline.score(X_train, y_train))
print("Test Accuracy :", pipeline.score(X_test, y_test))

# --- Prediction helper ---
class_labels = {
    0: "Stable",
    1: "High Risk",
    2: "Landslide Event"
}

def predict_risk(input_dict):
    df_input = pd.DataFrame([input_dict])
    for col in X.columns:
        if col not in df_input:
            df_input[col] = np.nan
    df_input = df_input[X.columns]

    pred = pipeline.predict(df_input)[0]
    proba = pipeline.predict_proba(df_input)[0]

    pred_label = class_labels[pred]
    proba_dict = {class_labels[c]: float(p) for c, p in zip(pipeline.classes_, proba)}
    return pred_label, proba_dict

# --- Example test ---
sample_input = {
    # Replace with actual feature names and example values from your dataset
    X.columns[0]: 12.3,
    X.columns[1]: 45.6,
    X.columns[2]: 78.9,
    # etc...
}

pred, proba = predict_risk(sample_input)
print("Prediction:", pred)
print("Probabilities:", proba)


✅ Model trained
Train Accuracy: 0.91875
Test Accuracy : 0.87525
Prediction: Landslide Event
Probabilities: {'Stable': 0.0, 'High Risk': 0.005, 'Landslide Event': 0.995}


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
import joblib

# --- Step 1: Load the saved pipeline ---
pipeline = joblib.load("rf_landslide_classifier.pkl")
print("✅ Model loaded successfully!")

# --- Step 2: Define class labels ---
class_labels = {
    0: "Stable",
    1: "High Risk",
    2: "Landslide Event"
}

# --- Step 3: Prediction helper function ---
def predict_risk(input_dict):
    df_input = pd.DataFrame([input_dict])

    # Ensure all features are present
    for col in pipeline.named_steps['imputer'].feature_names_in_:
        if col not in df_input:
            df_input[col] = np.nan

    df_input = df_input[pipeline.named_steps['imputer'].feature_names_in_]

    pred = pipeline.predict(df_input)[0]
    proba = pipeline.predict_proba(df_input)[0]

    pred_label = class_labels[pred]
    proba_dict = {class_labels[c]: p for c, p in zip(pipeline.classes_, proba)}

    return pred_label, proba_dict

# --- Step 4: Example input data ---
example_input = {
    "temperature_C": 14.868,
    "rainfall_mm": 6.946,
    "pitch_deg": 2.4122,
    "soil_moisture": 0.22,
    "roll_deg": 0.9999,
    "crack_mm": 0.49916,
    "crack_growth_mm_per_hr": 0.00017,
    "accel_x_g": -0.03961,
    "accel_y_g": 0.01297,
    "accel_z_g": 0.99266,
    "vibration_rms_g": 0.04279,
    "vibration_peak_g": 0.02472
}

# --- Step 5: Make prediction ---
prediction, probabilities = predict_risk(example_input)
print("\nExample Prediction:")
print("Predicted Class:", prediction)
print("Class Probabilities:", probabilities)


✅ Model loaded successfully!

Example Prediction:
Predicted Class: High Risk
Class Probabilities: {'Stable': np.float64(0.1874943912054663), 'High Risk': np.float64(0.6624961462947841), 'Landslide Event': np.float64(0.15000946249974956)}


In [8]:
# Saving landslide_merged_demo.csv to landslide_merged_demo.csv
# Train Accuracy: 0.91875
# Test Accuracy : 0.87525
# ✅ Model saved as rf_landslide_classifier.pkl

# Example Prediction:
# Predicted Class: High Risk
# Class Probabilities: {'Stable': np.float64(0.1874943912054663), 'High Risk': np.float64(0.6624961462947841), 'Landslide Event': np.float64(0.15000946249974956)}

curl -X POST http://voidpanel.com/predict/ \
     -H "Content-Type: application/json" \
     -d '{
        "temperature_C": 14.868,
        "rainfall_mm": 6.946,
        "pitch_deg": 2.4122,
        "soil_moisture": 0.22,
        "roll_deg": 0.9999,
        "crack_mm": 0.49916,
        "crack_growth_mm_per_hr": 0.00017,
        "accel_x_g": -0.03961,
        "accel_y_g": 0.01297,
        "accel_z_g": 0.99266,
        "vibration_rms_g": 0.04279,
        "vibration_peak_g": 0.02472
     }'


In [9]:
# ======================================================
#   LANDSLIDE RISK ML MODEL - UNIFIED STRUCTURE
#   Works for: Colab, Local, Django, API, Batch Prediction
# ======================================================

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os

# ======================================================
#   CONFIG
# ======================================================
MODEL_PATH = "rf_landslide_classifier.pkl"
CSV_PATH = "landslide_merged_demo.csv"      # put your dataset here

TARGET = "label"

class_labels = {
    0: "Stable",
    1: "High Risk",
    2: "Landslide Event"
}

# ======================================================
#   1. TRAIN THE MODEL
# ======================================================
def train_model(csv_path=CSV_PATH, save=True):
    print("📌 Loading CSV:", csv_path)
    df = pd.read_csv(csv_path)

    X = df.drop(columns=["timestamp", TARGET])
    y = df[TARGET]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
        ("model", RandomForestClassifier(
            n_estimators=200,
            max_depth=12,
            random_state=42,
            class_weight="balanced"
        ))
    ])

    pipeline.fit(X_train, y_train)

    print("\n🎯 ACCURACY")
    print("Train Accuracy:", pipeline.score(X_train, y_train))
    print("Test Accuracy :", pipeline.score(X_test, y_test))

    if save:
        joblib.dump(pipeline, MODEL_PATH)
        print(f"\n💾 Model saved as {MODEL_PATH}")

    return pipeline, X.columns.tolist()


# ======================================================
#   2. LOAD SAVED MODEL
# ======================================================
def load_model(model_path=MODEL_PATH):
    if not os.path.exists(model_path):
        raise FileNotFoundError("❌ Model not found. Train it first.")

    pipeline = joblib.load(model_path)
    feature_names = pipeline.named_steps["imputer"].feature_names_in_
    print("✅ Model loaded successfully")
    return pipeline, feature_names


# ======================================================
#   3. SINGLE PREDICTION HELPER
# ======================================================
def predict_risk(input_dict, pipeline=None, feature_names=None):

    if pipeline is None:
        pipeline, feature_names = load_model()

    df_input = pd.DataFrame([input_dict])

    # Ensure missing columns are added
    for col in feature_names:
        if col not in df_input:
            df_input[col] = np.nan

    df_input = df_input[feature_names]  # maintain order

    pred = pipeline.predict(df_input)[0]
    proba = pipeline.predict_proba(df_input)[0]

    pred_label = class_labels[pred]
    proba_dict = {class_labels[c]: float(p) for c, p in zip(pipeline.classes_, proba)}

    return pred_label, proba_dict


# ======================================================
#   4. OPTIONAL: QUICK TEST
# ======================================================
if __name__ == "__main__":
    # Train the model (only run once)
    pipeline, features = train_model()

    # Example input
    example_input = {
        "temperature_C": 14.868,
        "rainfall_mm": 6.946,
        "pitch_deg": 2.4122,
        "soil_moisture": 0.22,
        "roll_deg": 0.9999,
        "crack_mm": 0.49916,
        "crack_growth_mm_per_hr": 0.00017,
        "accel_x_g": -0.03961,
        "accel_y_g": 0.01297,
        "accel_z_g": 0.99266,
        "vibration_rms_g": 0.04279,
        "vibration_peak_g": 0.02472
    }

    pred, proba = predict_risk(example_input, pipeline, features)
    print("\n🔮 Prediction:", pred)
    print("📊 Probabilities:", proba)


📌 Loading CSV: landslide_merged_demo.csv

🎯 ACCURACY
Train Accuracy: 0.91875
Test Accuracy : 0.87525

💾 Model saved as rf_landslide_classifier.pkl

🔮 Prediction: High Risk
📊 Probabilities: {'Stable': 0.1874943912054663, 'High Risk': 0.6624961462947841, 'Landslide Event': 0.15000946249974956}
